In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.insert(0, os.path.abspath("../")) 

import cv2
import numpy as np

from strikepoint.frames import FrameInfoReader
from IPython.display import Image, display


def showFrame(frame):
    ok, buf = cv2.imencode('.gif', frame)
    if not ok:
        raise RuntimeError("Could not encode frame to GIF")
    display(Image(data=buf.tobytes()))

with FrameInfoReader("data/demo-three-hits.bin") as reader:
    frameInfoList = list(enumerate(reader.readAllFrameInfo()))


In [ ]:
from strikepoint.frames import FrameInfoReader
from strikepoint.imaging import findBrightestCircles, drawBrightestCircles

# Affine transform from calibration, hardcoded here for demo purposes
M_Therm2Vis = np.array([[1.38724782e+00, 2.01100450e-02, -7.83809764e+01],
                        [3.28716025e-02, 1.40602086e+00, -6.26867910e+01]])
foundHistory, foundSeq = list(), tuple([True]*3 + [False]*3)

for i, frameInfo in frameInfoList[::]:
    visFrame = frameInfo.rgbFrames['visual']
    Hv, Wv = visFrame.shape[:2]
    visCircles = findBrightestCircles(visFrame, 1, throwOnFail=False)
    foundSingleCircle = len(visCircles) == 1
    visCircle = visCircles[0] if foundSingleCircle else None
    foundHistory.append((frameInfo, foundSingleCircle, visCircle))

    if len(foundHistory) == len(foundSeq):
        if all(a == b[1] for a, b in zip(foundSeq, foundHistory)):
            dt = foundHistory[-1][0].timestamp - foundHistory[0][0].timestamp
            t1 = foundHistory[0][0].rgbFrames['thermal']
            t2 = foundHistory[-1][0].rgbFrames['thermal']
            v1 = foundHistory[0][0].rgbFrames['visual']
            c = foundHistory[0][2]

            # Based on the foundHistory config, build an average of the
            # frames before and after the ball disappears from the frame
            beforeFrames = list(a[0].rawFrames['thermal']
                                for a in foundHistory if a[1])
            afterFrames = list(a[0].rawFrames['thermal']
                               for a in foundHistory if not a[1])
            thermalDiff = ((sum(afterFrames) / len(afterFrames)) -
                           (sum(beforeFrames) / len(beforeFrames)))

            # Clean and clip the image so we only see POSITIVE heat delta.  In
            # scenarios where a ball is warmer than the scene, this is required
            thermalDiff = cv2.flip(thermalDiff, 0)  # TODO: Shouldn't need this
            thermalDiff = cv2.flip(thermalDiff, 1)  # TODO: Shouldn't need this
            thermalDiff = np.clip(thermalDiff, thermalDiff.max()*0.1, None)
            thermalDiff = cv2.GaussianBlur(thermalDiff, (5, 5), 0)
            thermalDiff = cv2.resize(thermalDiff, t1.shape[:2][::-1],
                                     interpolation=cv2.INTER_NEAREST)
            thermalDiff = cv2.normalize(
                thermalDiff, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
            thermalDiff = cv2.applyColorMap(thermalDiff, cv2.COLORMAP_HOT)
            thermalDenoised = cv2.fastNlMeansDenoising(
                thermalDiff, None, h=50, templateWindowSize=7,
                searchWindowSize=21)

            thermalDiffW = cv2.warpAffine(
                thermalDiff, M_Therm2Vis, (Wv, Hv), flags=cv2.INTER_LINEAR,
                borderMode=cv2.BORDER_CONSTANT, borderValue=0)
            thermalDenoisedW = cv2.warpAffine(
                thermalDenoised, M_Therm2Vis, (Wv, Hv), flags=cv2.INTER_LINEAR,
                borderMode=cv2.BORDER_CONSTANT, borderValue=0)

            final = cv2.add(v1, thermalDenoisedW)
            diffGray = cv2.cvtColor(thermalDiffW, cv2.COLOR_RGB2GRAY)
            diffShape = tuple(final.shape[:2][::-1])
            leftDiff, rightDiff = diffGray.copy(), diffGray.copy()
            cv2.rectangle(rightDiff, (0, 0), (c[0]-c[2], diffShape[1]), 0, -1, 1)
            cv2.rectangle(leftDiff, (c[0]+c[2], 0), diffShape, 0, -1, 1)
            leftScore = leftDiff.sum() / diffGray.sum()
            rightScore = rightDiff.sum() / diffGray.sum()
            print(f"Frame #{i}, sequence {dt*1000:.1f}s")
            print(f"Left: {100*leftScore:.1f}%, Right: {100*rightScore:.1f}%")

            cv2.circle(
                thermalDiffW, (int(c[0]), int(c[1])), c[2], (0, 255, 0), 2)
            cv2.circle(final, (int(c[0]), int(c[1])), c[2], (0, 255, 0), 2)
            showFrame(np.hstack((final, thermalDiffW)))

        foundHistory = foundHistory[1:]